<a href="https://colab.research.google.com/github/Ni-cole17/Oficina-SABIO/blob/main/Oficina_SABIO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importações

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.2/645.2 kB 6.2 MB/s eta 0:00:00


In [5]:
import os
import cv2
import shutil
from ultralytics import YOLO

* os: A biblioteca os fornece funções para interagir com o sistema operacional.
Ela é útil para realizar operações como acessar o sistema de arquivos, criar diretórios, gerenciar variáveis de ambiente, e etc.

* cv2 (OpenCV): O OpenCV é uma biblioteca amplamente usada para processamento de imagens e visão computacional. Ele oferece uma variedade de funcionalidades, incluindo a leitura e escrita de imagens, detecção de objetos, rastreamento de objetos, manipulação de vídeo e etc.

* shutil: A biblioteca shutil é usada para operações de alto nível relacionadas à manipulação de arquivos e diretórios. Ela permite copiar, mover, renomear e excluir arquivos e diretórios de forma eficiente.

* ultralytics.YOLO: A biblioteca Ultralytics é uma API para trabalhar com detecção de objetos usando a arquitetura YOLO (You Only Look Once). Ela oferece um conjunto de ferramentas para treinar, avaliar e usar modelos YOLO para tarefas de detecção de objetos em imagens e vídeos.

# Base de dados

## Carregando a base

In [6]:
!pwd

/content


In [8]:
## Link para download: dl.raabindata.com/WBC/nucleus_cytoplasm_GT/GrTh.rar
!unzip "/content/drive/MyDrive/Oficina - SABIO/database.zip"

Archive:  /content/drive/MyDrive/Oficina - SABIO/database.zip
replace Original/Basophil/20190526_163027_0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: Original/Basophil/20190526_163027_0.jpg  
  inflating: Original/Basophil/20190526_163414_0.jpg  
  inflating: Original/Basophil/20190526_163446_1.jpg  
  inflating: Original/Basophil/20190526_163532_0.jpg  
  inflating: Original/Basophil/20190526_163545_0.jpg  
  inflating: Original/Basophil/20190526_163608_1.jpg  
  inflating: Original/Basophil/20190526_163656_0.jpg  
  inflating: Original/Basophil/20190526_163721_0.jpg  
  inflating: Original/Basophil/20190526_163735_0.jpg  
  inflating: Original/Basophil/20190526_163753_0.jpg  
  inflating: Original/Basophil/20190526_163806_0.jpg  
  inflating: Original/Basophil/20190526_164040_0.jpg  
  inflating: Original/Basophil/20190526_164053_0.jpg  
  inflating: Original/Basophil/20190526_164136_0.jpg  
  inflating: Original/Basophil/20190526_164157_0.jpg  
  inflating: Original/Ba

## Verificando contagem de cada classe

In [9]:
## Declaração da lista classes para armazenar as classes do nosso dataset
classes = []
## Iteração na base de imagens original e print da quantidade de imagens por cada classe
for classe in os.listdir("Original"):
    classes.append(classe)
    print(f'{classe}: {len(os.listdir(os.path.join("Original",classe)))}')

Basophil: 218
Monocyte: 242
Neutrophil: 242
Lymphocyte: 242
Eosinophil: 201


## Split em treino/teste/validação

In [10]:
## Vamos escolher o split de (80% treino | 10% teste | 10% validação)

## Escolha da contagem da classe com menos representação
total = 201
## Definindo o tamanho de treino, teste e validação
train_size = int(80 * total / 100)
val_size = int(10 * total / 100)
test_size = total - train_size - val_size
## Printando os valores
print(train_size,val_size,test_size)

160 20 21


In [11]:
## Criando a base de dados no formato para "entregar" para o modelo
'''
database
|_train
    |_classe1
    |_classe2
    |_classe3
    ...
|_test
    |_classe1
    |_classe2
    |_classe3
    ...
|_val
    |_classe1
    |_classe2
    |_classe3
    ...
'''
for classe in classes:
    os.makedirs(os.path.join("database","train",classe),exist_ok = True)
    os.makedirs(os.path.join("database","test",classe),exist_ok = True)
    os.makedirs(os.path.join("database","val",classe),exist_ok = True)

In [12]:
## Movendo as imagens da base original para a base de dados do modelo
for classe in classes:
    for img in os.listdir(os.path.join("Original", classe)):
        if len(os.listdir(os.path.join("database","val",classe))) < val_size:
            shutil.copy(os.path.join("Original", classe, img),os.path.join("database","val",classe))
        elif len(os.listdir(os.path.join("database","train",classe))) < train_size:
            shutil.copy(os.path.join("Original", classe, img),os.path.join("database","train",classe))
        elif len(os.listdir(os.path.join("database","test",classe))) < test_size:
            shutil.copy(os.path.join("Original", classe, img),os.path.join("database","test",classe))

In [13]:
## Verificando a contagem de treino, teste e validação
for patch in os.listdir("database"):
    print(patch)
    for classe in os.listdir(os.path.join("database",patch)):
        print(f'{classe}: {len(os.listdir(os.path.join("database",patch,classe)))}')

val
Basophil: 20
Monocyte: 20
Neutrophil: 20
Lymphocyte: 20
Eosinophil: 20
train
Basophil: 160
Monocyte: 160
Neutrophil: 160
Lymphocyte: 160
Eosinophil: 160
test
Basophil: 21
Monocyte: 21
Neutrophil: 21
Lymphocyte: 21
Eosinophil: 21


# Definição e treino do modelo

Documentação: https://docs.ultralytics.com/modes/train/#usage-examples

In [14]:
model = YOLO('yolov8n-cls.pt') ## Modelo pré treinado da Imagenet

100%|██████████| 5.28M/5.28M [00:00<00:00, 101MB/s]


In [15]:
## Verificando o tamanho das imagens
img = cv2.imread("/content/Original/Eosinophil/95-5-10-1_178_2.jpg")
print(img.shape)

(575, 575, 3)


In [16]:
## Treino do modelo (Hiperparâmetros: nº de épocas, tamanho da imagem de entrada,...)
results = model.train(data = "database", epochs=10, imgsz=576)

Ultralytics YOLOv8.0.207 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=database, epochs=10, patience=50, batch=16, imgsz=576, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, 

In [17]:
## Validando o modelo
metrics = model.val(data = "database")

Ultralytics YOLOv8.0.207 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n-cls summary (fused): 73 layers, 1441285 parameters, 0 gradients, 3.3 GFLOPs
train: /content/database/train... found 800 images in 5 classes ✅ 
val: /content/database/val... found 100 images in 5 classes ✅ 
test: /content/database/test... found 105 images in 5 classes ✅ 
val: Scanning /content/database/val... 100 images, 0 corrupt: 100%|██████████| 100/100 [00:00<?, ?it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 7/7 [00:02<00:00,  2.66it/s]
                   all       0.98          1
Speed: 1.3ms preprocess, 8.7ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train2


## Métricas

In [18]:
metrics

ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a5c6f488700>
curves: []
curves_results: []
fitness: 0.9899999797344208
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.9799999594688416, 'metrics/accuracy_top5': 1.0, 'fitness': 0.9899999797344208}
save_dir: PosixPath('runs/classify/train2')
speed: {'preprocess': 1.250605583190918, 'inference': 8.699438571929932, 'loss': 0.0020575523376464844, 'postprocess': 0.0012660026550292969}
task: 'classify'
top1: 0.9799999594688416
top5: 1.0

## Predição

In [19]:
## Carregando o melhor modelo treinado
model = YOLO('/content/runs/classify/train/weights/best.pt')

In [20]:
## Inferência em uma imagem
results = model(["/content/database/train/Basophil/20190526_163027_0.jpg"])


0: 576x576 Basophil 1.00, Eosinophil 0.00, Monocyte 0.00, Neutrophil 0.00, Lymphocyte 0.00, 4.5ms
Speed: 5.7ms preprocess, 4.5ms inference, 0.1ms postprocess per image at shape (1, 3, 576, 576)


In [21]:
!cp -r "runs/" "/content/drive/MyDrive/Oficina - SABIO"